In [1]:
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from settings import *
from prp import *
from preprocess import *
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import cv2



In [2]:
# Load trained ProtoPNet model
load_model_path = os.path.join(load_model_dir, load_model_name)
ppnet = torch.load(load_model_path, map_location=torch.device(device))
prp_model = PRPCanonizedModel(ppnet, base_architecture)


/Users/kr21883/Library/CloudStorage/OneDrive-UniversityofBristol/Code/PRP_Label_Imbalance/myenv/lib/python3.9/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kr21883/Library/CloudStorage/OneDrive-UniversityofBristol/Code/PRP_Label_Imbalance/myenv/lib/python3.9/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kr21883/Library/CloudStorage/OneDrive-UniversityofBristol/Code/PRP_La

at src_module_name 


at src_module_name conv1
is Conv2d


at src_module_name bn1
is BatchNorm2d
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
conv2d_zbeta_wrapper_class(
  (module): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
)
zeroparam_wrapper_class(
  (module): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


at src_module_name relu


at src_module_name maxpool


at src_module_name layer1


at src_module_name layer1.0


at src_module_name layer1.0.conv1
is Conv2d


at src_module_name layer1.0.bn1
is BatchNorm2d
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
oneparam_wrapper_class(
  (module): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
zeroparam_wrapper_class(
  (module): BatchNorm2d(6

In [3]:
# For a single test image and for a particular prototype
test_image_path = os.path.join(test_image_dir, test_image_name)

image = Image.open(test_image_path)

# To match trained PRP model
# Convert the grayscale image to RGB
rgb_image = image.convert("RGB")
new_size = (224, 224)
new_image = rgb_image.resize(new_size)


img = torch.asarray(np.array(new_image).transpose([2, 0, 1])).float()
img_variable = Variable(img.unsqueeze(0))/255
img_tensor = preprocess(img_variable, mean, std)

images_test = img_variable.to(device)
prp_map = generate_prp_image(images_test, prototype_number, prp_model, device)
makedir(write_path)
plt.imsave(write_path+"prp_"+str(prototype_number)+"_" +
           test_image_name, prp_map, cmap="seismic", vmin=-1, vmax=+1)



#############
### OVERLAY ##########
################
def invert_normalize(ten, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    print(ten.shape)
    s = torch.tensor(np.asarray(std, dtype=np.float32)
                     ).unsqueeze(1).unsqueeze(2)
    m = torch.tensor(np.asarray(mean, dtype=np.float32)
                     ).unsqueeze(1).unsqueeze(2)

    res = ten * s + m
    return res


heatmap = cv2.imread("Test images/PRP/"+"prp_" +
                     str(prototype_number)+"_"+test_image_name)
heatmap = heatmap[..., ::-1]
heatmap = np.float32(heatmap) / 255
ts = invert_normalize(img_tensor.squeeze())
a = ts.data.numpy().transpose((1, 2, 0))
overlayed_original_img_j = 0.2 * a + 0.6 * heatmap
plt.imsave("Test images/PRP/"+"Overlay_prp_"+str(prototype_number)+"_"+test_image_name,
           overlayed_original_img_j,
           vmin=-1,
           vmax=+1.0)

maxpool2d custom forward


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


l2 custom forward
maxpool2d custom forward
l2 custom backward


torch.Size([3, 224, 224])
